In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd

data = pd.read_excel(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls"
)
# rename
data.rename({
    c: 'Z_{}'.format(i) for c, i in zip(data.columns, range(len(data.columns)))
}, axis=1, inplace=True)

# normalize
for c in data.columns:
    data[c] = (data[c] - data[c].min()) / (data[c].max() - data[c].min())
gt = data.values

In [6]:
from sklearn.impute import SimpleImputer
from matplotlib import pyplot as plt
import numpy as np
from hyperimpute.plugins.imputers import Imputers

range_ = np.linspace(0, 1, 3)
rmse = []

for it in range(10):
    print(it)
    r = 0.3
    mask = np.random.choice([np.nan, 1], p=[r, 1-r], size=data.shape)
    ds = np.multiply(gt, mask)
    n = np.isnan(mask).sum()
    hpi = Imputers().get(
        'hyperimpute',
        optimizer='hyperband',
        classifier_seed=['logistic_regression'],
        regression_seed=['linear_regression']
    )
    imp = hpi.fit_transform(pd.DataFrame(ds)).values
    sq = np.square(
        gt[np.isnan(mask)] - imp[np.isnan(mask)]
    )
    ssq = sq.sum()/n
    sqr = np.sqrt(ssq)
    rmse.append(sqr)

print(np.mean(rmse))
print(np.std(rmse))

0
1
2
3
4
5
6
7
8
9
0.15757568294278496
0.0031004201894142516
